## Getting text from PDF

In [1]:
# !pip install langchain openai

In [2]:

!pip install transformers faiss-gpu datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00


In [3]:
!pip install transformers torch datasets


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [4]:
!pip install PyMuPDF pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 17.6 MB/s eta 0:00:00


In [5]:
!pip install PyPDF2 pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:00


In [6]:
# Install tesseract-ocr in the Colab environment
!apt-get update
!apt-get install -y tesseract-ocr

# Optional: Install language packages
# For example, to install the English language package
!apt-get install -y tesseract-ocr-eng

# You might also need to install pytesseract if you haven't already
!pip install pytesseract

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [872 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,395 kB]
Get:13 http://archive.ubuntu.com/

In [7]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 23.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0


In [8]:
# # Setting up the authentication token
# from huggingface_hub import notebook_login

# notebook_login()

In [9]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import io
from concurrent.futures import ThreadPoolExecutor

def extract_text_or_ocr(page):
    text = page.get_text()
    if len(text.strip()) < 50:  # Adjust the threshold as needed
        pix = page.get_pixmap()
        image_bytes = pix.tobytes("png")
        image = Image.open(io.BytesIO(image_bytes))
        # Specify UTF-8 encoding for pytesseract output
        text = pytesseract.image_to_string(image, lang='eng', config='--psm 3 -c outputbase_charset=UTF-8')
    return text

def pdfs_to_text_optimized(pdf_paths):
    combined_text = []
    for pdf_path in pdf_paths:
        with fitz.open(pdf_path) as doc:
            with ThreadPoolExecutor() as executor:
                results = list(executor.map(extract_text_or_ocr, doc))
            combined_text.append("\n".join(results))
    final_text = "\n".join(combined_text)
    # Ensure the final text is treated as UTF-8
    return final_text.encode('utf-8').decode('utf-8', 'replace')

# Example usage for multiple PDFs
pdf_files = ['_COTB_StratPlan_2023_OnePager_11x8.5_v5---FINAL-STRAT-PLAN.pdf', ]
large_document_text = pdfs_to_text_optimized(pdf_files)


In [10]:
large_document_text

' \n \nMaamawe, \nGrowing Together \nCity of Thunder Bay \nStrategic Plan 2023-2027 \nJULY 2023 \n\n \n \n \n \n  \n \nMaamawe, growing together. \nO\nur \nVision \nAll together (Maamawe), Thunder Bay will foster an inclusive city focused \non service excellence and collaboration, and provide opportunities for a high \nquality of life. We embrace and celebrate our diversity, which makes our \ncommunity a vibrant and dynamic place to live and prosper. \nOur Values \n• Accountability \n• Continuous Improvement \n \n• Teamwork \n \n• Respect \n= A.C.T. with Respect \nOur Promise \nWe will improve the quality of life for people \n \nin Thunder Bay through tangible gains in: \n \n• Truth and reconciliation \n• Safety and wellbeing \n• Prosperity \n \n• Sustainability \n\n \n \n \n \n \n \n \n \n \n \n \n \n  \n  \n \n \n \n \n \n \n \n \n \n\xa0\n \n \n \n \n \n \n \n \n \n \n \n \n  \n \n \n \n \n  \n \n \n \nGOAL: Strengthen the City’s relationships \nwith Indigenous communities, leaders 

In [11]:
!ls

berti.pdf							new_question_bank.json
_COTB_StratPlan_2023_OnePager_11x8.5_v5---FINAL-STRAT-PLAN.pdf	sample_data


##Get context

In [12]:
from transformers import AutoTokenizer, AutoModel, AutoModelForQuestionAnswering
import faiss
import numpy as np
from datasets import Dataset
import torch

In [13]:
# Initialize tokenizer and model
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
context_model = AutoModel.from_pretrained(model_name)

def split_document(text, chunk_size=512 - tokenizer.num_special_tokens_to_add()):
    """
    Splits the document into manageable chunks considering special tokens.
    """
    tokens = tokenizer(text, add_special_tokens=False, return_tensors="pt")['input_ids'][0]
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]
    chunk_texts = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks if len(chunk) > 0]
    return chunk_texts


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [14]:
# def encode_chunks(chunk_texts):
#     """
#     Encodes text chunks into embeddings, ensuring proper tokenization and handling.
#     """
#     context_model.eval()
#     embeddings = []
#     for text in chunk_texts:
#         if text.strip():  # Only process non-empty text
#             inputs = tokenizer(text, return_tensors='pt', padding="max_length", truncation=True, max_length=512)
#             with torch.no_grad():
#                 outputs = context_model(**inputs)
#                 chunk_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
#             embeddings.append(chunk_embedding)
#     return np.array(embeddings)

In [15]:
# dimension = context_model.config.hidden_size
# index = faiss.IndexFlatL2(dimension)

# def query_index_for_text(query_text, chunks, top_k=1):
#     query_tokens = tokenizer(query_text, return_tensors="pt", padding=True, truncation=True)
#     with torch.no_grad():
#         query_embedding = context_model(**query_tokens).last_hidden_state.mean(dim=1).cpu().numpy()
#     distances, indices = index.search(query_embedding, top_k)
#     relevant_chunks_text = [chunks[idx] for idx in indices[0] if idx < len(chunks)]
#     return relevant_chunks_text[0] if relevant_chunks_text else ""

In [16]:

# # Example usage
# chunks = split_document(large_document_text)
# embeddings = encode_chunks(chunks)
# index.add(embeddings)

# test_query_text = "From the context given, do they have a goal of enhancing safety and well-being at the community level"
# test_relevant_context = query_index_for_text(test_query_text, chunks, top_k=1)
# print(test_relevant_context)

In [17]:
# test_relevant_context

In [18]:
# index =[]
# dimension = context_model.config.hidden_size  # Dimension of embeddings
# index = faiss.IndexFlatL2(dimension)

# chunks = split_document(large_document_text)
# embeddings = encode_chunks(chunks)
# index.add(embeddings)

# query_text = "What is the vision of the company"
# relevant_context = query_index_for_text(query_text, chunks, top_k=1)

In [19]:
# relevant_context

In [20]:
def normalize_embeddings(embeddings):
    """ Normalize the embeddings to unit length for cosine similarity """
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

def split_document(text, chunk_size=512 - tokenizer.num_special_tokens_to_add()):
    tokens = tokenizer(text, add_special_tokens=False, return_tensors="pt")['input_ids'][0]
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]
    chunk_texts = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks if len(chunk) > 0]
    return chunk_texts

def encode_chunks(chunk_texts):
    context_model.eval()
    embeddings = []
    for text in chunk_texts:
        if text.strip():
            inputs = tokenizer(text, return_tensors='pt', padding="max_length", truncation=True, max_length=512)
            with torch.no_grad():
                outputs = context_model(**inputs)
                chunk_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
            embeddings.append(chunk_embedding)
    return normalize_embeddings(np.array(embeddings))

# Initialize FAISS index with cosine similarity
dimension = context_model.config.hidden_size
index = faiss.IndexFlatIP(dimension)  # Inner Product (cosine similarity)

def query_index_for_text(query_text, chunks, top_k=1):
    query_tokens = tokenizer(query_text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        query_embedding = context_model(**query_tokens).last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    query_embedding = normalize_embeddings(query_embedding.reshape(1, -1))  # Normalize the query embedding
    distances, indices = index.search(query_embedding, top_k)
    relevant_chunks_text = [chunks[idx] for idx in indices[0] if idx < len(chunks)]
    return relevant_chunks_text[0] if relevant_chunks_text else ""


In [21]:
# Example usage
chunks = split_document(large_document_text)
embeddings = encode_chunks(chunks)
index.add(embeddings)

test_query_text = "From the context given, do they have a goal of enhancing safety and well-being at the community level"
test_relevant_context = query_index_for_text(test_query_text, chunks, top_k=3)
print(test_relevant_context)

Token indices sequence length is longer than the specified maximum sequence length for this model (1280 > 512). Running this sequence through the model will result in indexing errors


survey : decrease in positive response re : experiences with discrimination. 4. citizen satisfaction survey : % agree racism is a serious issue in thunder bay. key performance indicators : safety and well - being. our community is healthy, safe, and strong. goal : improve access to supports for priority populations to narrow gaps in equity. objectives : a. advocate for provincial funding to recognize our city as a regional hub to enhance health and social services to meet our community ’ s needs. b. support harm reduction by decriminalizing people who use drugs. c. create opportunities for equitable access to affordable recreation, culture, transportation, and quality health care across the continuum. goal : enhance safety and well - being at the community level through climate action and environmental design. objectives : a. deliver public education on climate action and climate resilience opportunities as identified in the net zero strategy. b. implement action plans to advance crime

In [22]:
chunks = split_document(large_document_text)
embeddings = encode_chunks(chunks)
index.add(embeddings)

test_query_text = "Objective of the company"
test_relevant_context = query_index_for_text(test_query_text, chunks, top_k=1)
print(test_relevant_context)

maamawe, growing together city of thunder bay strategic plan 2023 - 2027 july 2023 maamawe, growing together. o ur vision all together ( maamawe ), thunder bay will foster an inclusive city focused on service excellence and collaboration, and provide opportunities for a high quality of life. we embrace and celebrate our diversity, which makes our community a vibrant and dynamic place to live and prosper. our values • accountability • continuous improvement • teamwork • respect = a. c. t. with respect our promise we will improve the quality of life for people in thunder bay through tangible gains in : • truth and reconciliation • safety and wellbeing • prosperity • sustainability goal : strengthen the city ’ s relationships with indigenous communities, leaders and organizations to advance truth & reconciliation priorities together. objectives : a. empower all city departments to engage in formal and informal relationships with indigenous organizations to identify shared priorities and a

## Train on context

In [23]:
import json

def process_data(file_path):
    if file_path.endswith('.json'):
      # Read the JSON file back into a Python list to verify
      with open(file_path, 'r') as json_file:
          loaded_data = json.load(json_file)

    return loaded_data

# Assuming 'file_path' is the path to the uploaded dataset file
question_bank_path = 'new_question_bank.json'
cleaned_data = process_data(question_bank_path)
print(cleaned_data)  # Optionally print or use the cleaned data as needed


[{'context': 'The Community Official Plan consists of the following text and the attached Schedules A to D3 (maps). The text of the Community Official Plan is organized into five (5) sections: 1. Introduction: contains the administrative components of the Plan; 2. Basis of the Plan: sets out the vision of the community, planning history and growth and settlement strategy; 3. Land Use Policies: details the specific policies for the various land uses recognized by the Plan; 4. General Policies: applies to all development within the Municipality; and, 5. Implementation: identifies the various planning tools available to implement the policies of the Plan, relevant legislation and definitions.', 'question': 'Does the Community Official Plan include maps in its documentation?', 'label': 'Yes'}, {'context': 'The boundaries of the land use designations, natural features and constraints shown on Schedules A to D3 and Appendix A1-A2 are intended to be approximate. Boundaries are to be considere

In [24]:
from collections import Counter

print(Counter(entry["label"] for entry in cleaned_data))


Counter({'No': 376, 'Yes': 246, 'Maybe': 156})


In [25]:
cleaned_data[0]

{'context': 'The Community Official Plan consists of the following text and the attached Schedules A to D3 (maps). The text of the Community Official Plan is organized into five (5) sections: 1. Introduction: contains the administrative components of the Plan; 2. Basis of the Plan: sets out the vision of the community, planning history and growth and settlement strategy; 3. Land Use Policies: details the specific policies for the various land uses recognized by the Plan; 4. General Policies: applies to all development within the Municipality; and, 5. Implementation: identifies the various planning tools available to implement the policies of the Plan, relevant legislation and definitions.',
 'question': 'Does the Community Official Plan include maps in its documentation?',
 'label': 'Yes'}

In [26]:
import random
import json

def split_data(file_path, train_ratio=0.75):
    # Load the combined data from the JSON file
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)

    random.seed(42)
    # Shuffle the data to ensure random distribution
    random.shuffle(data)

    # Calculate the split index
    split_index = int(len(data) * train_ratio)

    # Split the data into training and evaluation sets
    training_data = data[:split_index]
    eval_data = data[split_index:]

    return training_data, eval_data

# Specify the path to the JSON file
training_data, eval_data = split_data(question_bank_path)

# Optionally print lengths or save them to new JSON files
print(f"Training Data Length: {len(training_data)}")
print(f"Evaluation Data Length: {len(eval_data)}")

# Save to JSON files if needed
with open('training_data.json', 'w') as file:
    json.dump(training_data, file, indent=4)

with open('eval_data.json', 'w') as file:
    json.dump(eval_data, file, indent=4)


Training Data Length: 583
Evaluation Data Length: 195


In [27]:
data = training_data

In [28]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch

In [29]:
# # model_name = "bert-base-uncased"
# model_name = "distilbert-base-uncased"
# # model_name = "albert-base-v2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load a pre-trained model for sequence classification instead of question answering
model_2 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # Assuming labels are Yes: 0, No: 1, Maybe: 2

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from datasets import Dataset
from transformers import AutoTokenizer


# Assuming 'data' is your initial data list of dictionaries
structured_data = {
    "context": [item["context"] for item in data],
    "question": [item["question"] for item in data],
    "label": [item["label"] for item in data],  # These are textual labels: 'Yes', 'No', 'Maybe'
}

# Convert the structured data into a Hugging Face Dataset
dataset = Dataset.from_dict(structured_data)

# Define a preprocessing function to tokenize your data
def preprocess_function(examples):
    # Tokenize context and question pairs
    tokenized_examples = tokenizer(examples["context"], examples["question"], padding="max_length", truncation=True, max_length=512)
    return tokenized_examples

# Apply the preprocessing function to tokenize the dataset
dataset = dataset.map(preprocess_function, batched=True)

# Define a function to convert textual labels ('Yes', 'No', 'Maybe') to numeric
def transform_labels(examples):
    label_dict = {"Yes": 0, "No": 1, "Maybe": 2}
    # Convert each textual label to its corresponding numeric label
    examples["labels"] = [label_dict[label] for label in examples["label"]]
    return examples

# Apply the label transformation to the dataset
dataset = dataset.map(transform_labels, batched=True)

# Remove the original 'label' field to avoid confusion, keeping only the numeric 'labels'
dataset = dataset.remove_columns(["label"])

Map:   0%|          | 0/583 [00:00<?, ? examples/s]

Map:   0%|          | 0/583 [00:00<?, ? examples/s]

In [31]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    logits, labels = p
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [32]:
eval_data = eval_data

In [33]:
# Convert the eval_data into a Hugging Face Dataset
from datasets import Dataset

# Convert the structured eval_data into a Hugging Face Dataset
eval_dataset = Dataset.from_dict({
    "context": [item["context"] for item in eval_data],
    "question": [item["question"] for item in eval_data],
    "label": [item["label"] for item in eval_data],  # These are textual labels: 'Yes', 'No', 'Maybe'
})


# Preprocess eval_dataset similar to how the training dataset was processed
eval_dataset = eval_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(transform_labels, batched=True)

# Remove the original 'label' field to avoid confusion, keeping only the numeric 'labels'
eval_dataset = eval_dataset.remove_columns(["label"])

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

In [34]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=20,             # Maximum number of training epochs
    per_device_train_batch_size=8,   # Batch size per device during training
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Weight decay if applicable
    logging_dir='./logs',            # Directory for logs
    logging_strategy="steps",        # Log metrics at regular steps
    logging_steps=10,                # Number of steps at which to log training metrics
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save checkpoints at the end of each epoch
    load_best_model_at_end=True,     # Load the best model at the end of training
    metric_for_best_model="eval_loss",  # Specify the metric to use for early stopping
)


from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(p):
    logits, labels = p
    predictions = np.argmax(logits, axis=-1)
    # For multi-class classification, use 'macro', 'micro', or 'weighted'
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

# Update the Trainer with the corrected compute_metrics function
trainer = Trainer(
    model=model_2,
    args=training_args,
    train_dataset=dataset,  # Assuming 'dataset' is your training dataset
    eval_dataset=eval_dataset,  # Assuming 'eval_dataset' is your evaluation dataset
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [35]:
# Now, you can start training
trainer.train() #number of epoches can be reduced or early stopping applied to stop overfitting

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.082100,1.084158,0.466667,0.212121,0.155556,0.333333
2,1.012100,1.053094,0.466667,0.212121,0.155556,0.333333
3,1.029300,1.044209,0.466667,0.212121,0.155556,0.333333
4,0.947100,1.043906,0.420513,0.330216,0.358446,0.375632
5,0.816300,0.870958,0.564103,0.543803,0.559906,0.558970
6,0.692200,0.854849,0.646154,0.569051,0.655130,0.572262
7,0.414800,0.913018,0.651282,0.608613,0.674326,0.594189
8,0.273800,1.103619,0.646154,0.613130,0.647164,0.608332
9,0.182400,1.044961,0.707692,0.701515,0.701865,0.702339
10,0.328900,1.206688,0.717949,0.708628,0.710475,0.707908


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1460, training_loss=0.3591187098905546, metrics={'train_runtime': 272.1285, 'train_samples_per_second': 42.847, 'train_steps_per_second': 5.365, 'total_flos': 386727131197440.0, 'train_loss': 0.3591187098905546, 'epoch': 20.0})

In [36]:
#evaluate

In [37]:
trainer.evaluate(eval_dataset)

{'eval_loss': 0.8548489809036255,
 'eval_accuracy': 0.6461538461538462,
 'eval_f1': 0.5690514905149051,
 'eval_precision': 0.655130249867092,
 'eval_recall': 0.5722618875161248,
 'eval_runtime': 1.1112,
 'eval_samples_per_second': 175.487,
 'eval_steps_per_second': 22.498,
 'epoch': 20.0}

In [38]:
# Ensure your model and tokenizer are properly initialized and loaded
# model_name = "albert-base-v2"  # or your trained model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained("./results")  # Load your trained model

# model.eval()

# Function to prepare inputs and get predictions
def predict(context, question, device='cuda'):
    # Move model to the specified device
    model_2.to(device)

    # Prepare inputs
    inputs = tokenizer(question, context, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Ensure model is in evaluation mode
    model_2.eval()

    # Perform prediction
    with torch.no_grad():
        outputs = model_2(**inputs)
    logits = outputs.logits

    # Move logits back to CPU for further processing if needed
    predictions = torch.softmax(logits, dim=-1).cpu()
    return predictions





In [39]:
# Example usage for multiple PDFs
pdf_files = ['berti.pdf', ]
large_document_text = pdfs_to_text_optimized(pdf_files)

In [40]:
# Check if CUDA is available and choose device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'


question = 'Does the study discuss background and relevant literature'

# get context
chunks = split_document(large_document_text)
embeddings = encode_chunks(chunks)
index.add(embeddings)

# Context and question
context =  query_index_for_text(question, chunks, top_k=1)


# Example usage of predict function with device specification
predictions = predict(context, question, device=device)


# Labels for your task
label_map = {0: "Yes", 1: "No", 2: "Maybe"}


# Get prediction
# predictions = predict(context, question)

# Get the highest probability label
predicted_label_idx = torch.argmax(predictions, dim=-1).item()
predicted_label = label_map[predicted_label_idx]

print(f"Question: {question}\nPredicted Answer: {predicted_label}")

Question: Does the study discuss background and relevant literature
Predicted Answer: No


In [41]:
context

': concept : name ” = cr. ” case : concept : name ” ), average throughput time as ( select avg ( throughput time ) as avg throughput from cases ) select avg ( case when c. throughput time < att. avg throughput then c. rework end ) as avg rework below average, avg ( case when c. throughput time > att. avg throughput then c. rework end ) as avg rework above average from cases as c, average throughput time as att correct response from gpt - 4. select case id, avg rework, avg throughput time, variant from ( select case id, avg ( rework ) as avg rework, avg ( duration ) as avg throughput time, concat ( activity ) as variant from dataframe group by case id ) as t where avg throughput time < > ( select avg ( duration ) from dataframe ) wrong response from bard. fig. 7 : sql query generation comparison between gpt - 4 and bard for the “ i want to check the average rework for cases below and above the average throughput time. ” query executed against the road traffic event log. • knowledge of s

In [42]:
# Check if CUDA is available and choose device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'


question = 'Did they study speak extensively about LLMs and process mining?'

# get context
chunks = split_document(large_document_text)
embeddings = encode_chunks(chunks)
index.add(embeddings)

# Context and question
context =  query_index_for_text(question, chunks, top_k=1)


# Example usage of predict function with device specification
predictions = predict(context, question, device=device)


# Labels for your task
label_map = {0: "Yes", 1: "No", 2: "Maybe"}


# Get prediction
# predictions = predict(context, question)

# Get the highest probability label
predicted_label_idx = torch.argmax(predictions, dim=-1).item()
predicted_label = label_map[predicted_label_idx]

print(f"Question: {question}\nPredicted Answer: {predicted_label}")

Question: Did they study speak extensively about LLMs and process mining?
Predicted Answer: No


In [43]:
context

'activities involving that particular object. this collation of graphs offers a nuanced view of the process, maintaining the sim - plicity of the traditional format. on the other hand, the object - centric petri net van der aalst and berti ( 2020 ) extends traditional petri nets to capture the lifecycle of each individual object type within the process. each object type is represented by its own petri net, and their integration forms the object - centric petri net. this detailed representation provides a comprehensive view of the process, accounting for the interactions between the multiple object springer nature 2021 l at ex template 7 types. these models provide a powerful means for analyzing complex, object - centric processes, enabling an in - depth understanding of the process from the perspective of individual object types and their interaction. 1. 3 declarative process models declarative process models provide a different perspective compared to the traditional procedural proces

In [44]:
# Check if CUDA is available and choose device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'


question = 'GPT-4 have shown remarkable progress in natural language processing tasks.'

# get context
chunks = split_document(large_document_text)
embeddings = encode_chunks(chunks)
index.add(embeddings)

# Context and question
context =  query_index_for_text(question, chunks, top_k=1)
print(f'Context: {context}')

# Example usage of predict function with device specification
predictions = predict(context, question, device=device)


# Labels for your task
label_map = {0: "Yes", 1: "No", 2: "Maybe"}


# Get prediction
# predictions = predict(context, question)

# Get the highest probability label
predicted_label_idx = torch.argmax(predictions, dim=-1).item()
predicted_label = label_map[predicted_label_idx]

print(f"Question: {question}\nPredicted Answer: {predicted_label}")

Context: abs / 2108. 07258 busch k, rochlitzer a, sola d, et al ( 2023 ) just tell me : prompt engineering in business process management. in : van der aa h, bork d, proper ha, et al ( eds ) enterprise, business - process and information systems modeling - 24th international conference, bpmds 2023, and 28th international conference, emmsad 2023, zaragoza, spain, june 12 - 13, 2023, proceedings, lecture notes in business information processing, vol 479. springer, pp 3 – 11, url https : / / doi. org / 10. 1007 / 978 - 3 - 031 - 34241 - 7 1 springer nature 2021 l at ex template 56 clark p, tafjord o, richardson k ( 2020 ) transformers as soft reasoners over language. in : bessiere c ( ed ) proceedings of the twenty - ninth international joint conference on artificial intelligence, ijcai 2020. ijcai. org, pp 3882 – 3890, url https : / / doi. org / 10. 24963 / ijcai. 2020 / 537 devlin j, chang m, lee k, et al ( 2019 ) bert : pre - training of deep bidirec - tional transformers for language 

In [45]:
# Check if CUDA is available and choose device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'


question = 'GPT-4 is the worst LLM ever!'

# get context
chunks = split_document(large_document_text)
embeddings = encode_chunks(chunks)
index.add(embeddings)

# Context and question
context =  query_index_for_text(question, chunks, top_k=1)
print(f'Context: {context}')

# Example usage of predict function with device specification
predictions = predict(context, question, device=device)


# Labels for your task
label_map = {0: "Yes", 1: "No", 2: "Maybe"}


# Get prediction
# predictions = predict(context, question)

# Get the highest probability label
predicted_label_idx = torch.argmax(predictions, dim=-1).item()
predicted_label = label_map[predicted_label_idx]

print(f"Question: {question}\nPredicted Answer: {predicted_label}")

Context: language. our primary focus lies on two prominent models - bard and gpt - 4 - and their ability to translate user - provided natural language queries into executable sql statements. furthermore, we are interested in how adept these models are at sponta - neously generating hypotheses based on an event log. the models are given basic abstractions such as the list of attributes and process variants within the log. for our analysis, we utilize duckdb sql language, a choice motivated by its seamless compatibility with the event log structure used by pm4py. in the present state of affairs, it is evident that some amount of domain knowledge is indispensable to ensure correct query formulation. this includes : springer nature 2021 l at ex template 41 i want to check the average rework for cases below and above the average throughput time. can you provide me a duckdb sql query. you should use the epoch function of duckdb to get the timestamp from the date. the data is stored in a pand

In [46]:
# Check if CUDA is available and choose device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'


question = 'Did the study talk about Julius Ceaser?'

# get context
chunks = split_document(large_document_text)
embeddings = encode_chunks(chunks)
index.add(embeddings)

# Context and question
context =  query_index_for_text(question, chunks, top_k=1)
print(f'Context: {context}')

# Example usage of predict function with device specification
predictions = predict(context, question, device=device)


# Labels for your task
label_map = {0: "Yes", 1: "No", 2: "Maybe"}


# Get prediction
# predictions = predict(context, question)

# Get the highest probability label
predicted_label_idx = torch.argmax(predictions, dim=-1).item()
predicted_label = label_map[predicted_label_idx]

print(f"Question: {question}\nPredicted Answer: {predicted_label}")

Context: who are successful, it can be worth the effort. here are some additional insights that can be drawn from the data : −the most common reason for appealing a fine is that the offender believes that the fine was unfair. −the most common outcome of an appeal is that the fine is upheld. −the success rate of appeals is relatively low. −the cost of appealing a fine can be significant. the data can be used to improve the process of appealing fines. for example, the data could be used to identify the factors that are most likely to lead to a successful appeal. the data could also be used to develop a more streamlined and efficient appeals process. springer nature 2021 l at ex template 34 listing 15 : root cause analysis on the bpi challenge 2020 event log by gpt - 4. from the process variants and the frequency and performance statistics provided, there are a few key steps that could be causing performance issues in the process : −rejection by administration or supervisor : variants whe

In [47]:
# Check if CUDA is available and choose device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'


question = 'LLMs have the potentials of performing process mining tasks'

# get context
chunks = split_document(large_document_text)
embeddings = encode_chunks(chunks)
index.add(embeddings)

# Context and question
context =  query_index_for_text(question, chunks, top_k=1)
print(f'Context: {context}')

# Example usage of predict function with device specification
predictions = predict(context, question, device=device)


# Labels for your task
label_map = {0: "Yes", 1: "No", 2: "Maybe"}


# Get prediction
# predictions = predict(context, question)

# Get the highest probability label
predicted_label_idx = torch.argmax(predictions, dim=-1).item()
predicted_label = label_map[predicted_label_idx]

print(f"Question: {question}\nPredicted Answer: {predicted_label}")

Context: inf syst 56 : 235 – 257. url https : / / doi. org / 10. 1016 / j. is. 2015. 07. 003 maggi fm, mooij aj, van der aalst wmp ( 2011 ) user - guided discovery of declarative process models. in : proceedings of the ieee symposium on computational intelligence and data mining, cidm 2011, part of the ieee symposium series on computational intelligence 2011, april 11 - 15, 2011, paris, france. ieee, pp 192 – 199, url https : / / doi. org / 10. 1109 / cidm. 2011. 5949297 de murillas egl, reijers ha, van der aalst wmp ( 2019 ) connecting databases with process mining : a meta model and toolset. softw syst model 18 ( 2 ) : 1209 – 1247 springer nature 2021 l at ex template 58 ouyang c, dumas m, ter hofstede ahm, et al ( 2006 ) from bpmn process models to bpel web services. in : 2006 ieee international conference on web services ( icws 2006 ), 18 - 22 september 2006, chicago, illinois, usa. ieee computer society, pp 285 – 292, url https : / / doi. org / 10. 1109 / icws. 2006. 67 pohl t, be